# 라벨 데이터 분석 코드


In [ ]:
from pathlib import Path
import cv2

import json

import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt

import matplotlib

matplotlib.rcParams['font.family'] = 'Malgun Gothic'


In [ ]:
with open('./of_part1.json') as json_data:
    data = json.load(json_data)
    image_df = pd.DataFrame(data['images'])
    annot_df = pd.DataFrame(data['annotations'])


In [ ]:
annot_df['width'] = annot_df['keypoints'].apply(lambda x: ((x[0] - x[2])**2 + (x[1] - x[3])**2)**0.5)

annot_df['height'] = annot_df['keypoints'].apply(lambda x: ((x[4] - x[6])**2 + (x[5] - x[7])**2)**0.5)


In [ ]:
annot_df['width'].mean(), annot_df['width'].median()


In [ ]:
annot_df['height'].mean(), annot_df['height'].median()


In [ ]:
annot_df['width'].var(), annot_df['height'].var()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(annot_df['width'], bins=30, kde=True, color='skyblue')
plt.title('Width 분포 히스토그램과 KDE 플롯')
plt.xlabel('Width')
plt.ylabel('빈도수 / 밀도')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(annot_df['height'], bins=30, kde=True, color='skyblue')
plt.title('Height 분포 히스토그램과 KDE 플롯')
plt.xlabel('Height')
plt.ylabel('빈도수 / 밀도')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=annot_df[['width', 'height']], palette='Set3')
plt.title('Width와 Height 분포 박스 플롯')
plt.xlabel('변수')
plt.ylabel('값')
plt.show()


# 주석 시각화 코드


In [ ]:
output_dir = Path('./of_part1_vis')
output_dir.mkdir(parents=True, exist_ok=True)

annot_grouped = annot_df.groupby('image_id')

for row in image_df.itertuples(index=False):
    image_id = row.id
    file_name = row.file_name

    image_path = Path('./of_part1') / Path(file_name).name
    print(f'Processing Image - Path: {image_path}, ID: {image_id}')

    img = cv2.imread(str(image_path))

    if img is None:
        print(f'Error: Failed to load image at {image_path}')
        continue  # 이미지 로드 실패 시 다음 이미지로 이동

    if image_id in annot_grouped.groups:
        annotations = annot_grouped.get_group(image_id)

        for annot in annotations.itertuples(index=False):
            bbox = annot.bbox
            if isinstance(bbox, list) and len(bbox) >= 4:
                x, y, w, h = bbox[:4]
                top_left = (int(x), int(y))
                bottom_right = (int(x + w), int(y + h))
                cv2.rectangle(img, top_left, bottom_right, (255, 0, 0), 2, cv2.LINE_AA)

            keypoints = annot.keypoints
            if isinstance(keypoints, list) and len(keypoints) >= 8:
                x1, y1, x2, y2 = keypoints[:4]
                cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2, cv2.LINE_AA)

                x3, y3, x4, y4 = keypoints[4:8]
                cv2.line(img, (int(x3), int(y3)), (int(x4), int(y4)), (0, 255, 0), 2, cv2.LINE_AA)

    output_path = output_dir / Path(file_name).name
    cv2.imwrite(str(output_path), img)
    print(f'Saved annotated image to {output_path}\n')


In [ ]:
image_df


In [ ]:
annot_df


In [ ]:
import pandas as pd
import os


def convert_bbox_to_yolo(size, bbox):
    """
    바운딩 박스를 YOLO 형식으로 변환합니다.

    Args:
        size (tuple): (width, height) 이미지의 크기
        bbox (list): [x, y, w, h]

    Returns:
        tuple: (x_center, y_center, w, h) 정규화된 값
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x, y, w, h = bbox
    x_center = x + w / 2.0
    y_center = y + h / 2.0
    return (x_center * dw, y_center * dh, w * dw, h * dh)


def convert_keypoints_to_yolo(size, keypoints):
    """
    키포인트를 YOLO 형식으로 정규화합니다.

    Args:
        size (tuple): (width, height) 이미지의 크기
        keypoints (list): [x1, y1, x2, y2, ..., xn, yn]

    Returns:
        list: 정규화된 키포인트 [x1_norm, y1_norm, x2_norm, y2_norm, ..., xn_norm, yn_norm]
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    normalized_keypoints = []
    for i in range(0, len(keypoints), 2):
        x_norm = keypoints[i] * dw
        y_norm = keypoints[i + 1] * dh
        normalized_keypoints.extend([x_norm, y_norm])
    return normalized_keypoints


def main():
    # DataFrame 불러오기
    with open('./of_part1.json') as json_data:
        data = json.load(json_data)
        image_df = pd.DataFrame(data['images'])
        annot_df = pd.DataFrame(data['annotations'])

    # bbox와 keypoints가 이미 리스트 형태로 되어 있다고 가정
    # 만약 리스트가 문자열 형태로 저장되어 있다면, 다음과 같이 변환할 수 있습니다.
    # import ast
    # annot_df['bbox'] = annot_df['bbox'].apply(ast.literal_eval)
    # annot_df['keypoints'] = annot_df['keypoints'].apply(ast.literal_eval)

    # 레이블을 저장할 디렉토리 생성
    labels_dir = './of_part1_labels1'  # 원하는 레이블 디렉토리 경로로 변경
    os.makedirs(labels_dir, exist_ok=True)

    # 클래스 ID 설정 (단일 클래스이므로 0)
    class_id = 0

    # 이미지 별로 레이블 파일 생성
    for _, image in image_df.iterrows():
        try:
            image_id = image['id']
            file_name = Path(image['file_name']).name
            img_width = image['width']
            img_height = image['height']

            # 해당 이미지의 애노테이션 필터링
            anns = annot_df[annot_df['image_id'] == image_id]

            if anns.empty:
                continue  # 애노테이션이 없는 경우 스킵

            label_lines = []
            for _, ann in anns.iterrows():
                bbox = ann['bbox']            # [x, y, w, h]
                keypoints = ann['keypoints']  # 리스트, 예: [x1, y1, x2, y2, ..., xn, yn]

                if not bbox or not keypoints:
                    continue  # bbox 또는 keypoints가 없는 경우 스킵

                try:
                    # YOLO bbox 변환
                    x_center, y_center, w, h = convert_bbox_to_yolo((img_width, img_height), bbox)
                except Exception as bbox_error:
                    print(f"Error converting bbox for image {image_id}: {bbox_error}")
                    continue

                try:
                    # YOLO keypoints 정규화
                    normalized_keypoints = convert_keypoints_to_yolo((img_width, img_height), keypoints)
                    keypoints_str = ' '.join([f"{kp:.6f}" for kp in normalized_keypoints])
                except Exception as kp_error:
                    print(f"Error converting keypoints for image {image_id}: {kp_error}")
                    continue

                # 한 줄의 레이블 생성 (단일 클래스이므로 class_id는 0)
                line = f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f} {keypoints_str}"
                label_lines.append(line)

            if not label_lines:
                continue  # 유효한 레이블이 없는 경우 스킵

            # 레이블 파일 경로 설정 (.txt 확장자)
            label_file = os.path.splitext(file_name)[0] + '.txt'
            label_path = os.path.join(labels_dir, label_file)

            # 레이블 파일 저장
            with open(label_path, 'w') as lf:
                lf.write('\n'.join(label_lines))

        except Exception as e:
            print(f"Error processing image {image_id}: {e}")


if __name__ == "__main__":
    main()


In [ ]:
with open('./datasets/infrared_of/label/json/000006.json', 'r') as f:
    data = json.load(f)

print(data['images'])


In [ ]:
import json


def cocoform2yolo(intput_path, output_path):

    # COCO JSON 데이터 불러오기

    with open(intput_path, 'r') as f:

        data = json.load(f)

    # 이미지 크기 정보 가져오기

    images = {img['id']: (img['width'], img['height']) for img in data['images']}

    # 바운딩 박스를 YOLO 형식으로 변환하는 함수

    def convert_bbox_to_yolo(bbox, img_width, img_height):

        x_min, y_min, x_max, y_max = bbox

        x_center = ((x_min + x_max) / 2) / img_width

        y_center = ((y_min + y_max) / 2) / img_height

        width = (x_max - x_min) / img_width

        height = (y_max - y_min) / img_height

        return x_center, y_center, width, height

    # 키포인트를 YOLO 형식으로 변환하는 함수

    def convert_keypoints_to_yolo(keypoints, img_width, img_height):

        yolo_keypoints = []

        for kp in keypoints:

            x, y = kp

            yolo_keypoints.extend([x / img_width, y / img_height, 2])

        return yolo_keypoints

    # YOLO 형식 변환 데이터를 저장할 딕셔너리

    yolo_data = {}

    for annotation in data['annotations']:

        img_id = annotation['image_id']

        class_type = annotation['type']

        bbox = annotation['bbox']

        keypoints = annotation['keypoints']

        img_width, img_height = images[img_id]

        # 바운딩 박스 변환

        x_center, y_center, width, height = convert_bbox_to_yolo(bbox, img_width, img_height)

        # 키포인트 변환

        yolo_keypoints = convert_keypoints_to_yolo(keypoints, img_width, img_height)

        # YOLO 형식의 라벨 줄 생성

        yolo_label = f"{int(class_type)-1} {x_center} {y_center} {width} {height} " + " ".join(map(str, yolo_keypoints))

        # 이미지 ID별로 YOLO 형식의 라벨 저장

        if img_id not in yolo_data:

            yolo_data[img_id] = []

        yolo_data[img_id].append(yolo_label)

    # YOLO 형식의 라벨을 텍스트 파일로 저장

    for img_id, labels in yolo_data.items():

        with open(f"{output_path}/{data['images'][0]['file_name'][:-4]}.txt", 'w') as f:

            f.write("\n".join(labels))


In [ ]:
from pathlib import Path

paths = Path('./datasets/infrared_of/globit_data/json/')
output_path = Path('./datasets/infrared_of/globit_data/labels/')


for input_path in paths.glob('*json'):
    cocoform2yolo(input_path, output_path)


In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2


def label_visualize(image_path, label_path):
    # 이미지 로드
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 로드할 수 없습니다: {image_path}")

    img_height, img_width, _ = image.shape
    print(img_height, img_width)

    # BGR을 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 레이블 파일 열기
    if not os.path.exists(label_path):
        print(f"레이블 파일이 존재하지 않습니다: {label_path}")

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            print(f"잘못된 레이블 형식: {line}")
            continue

        class_id = int(parts[0])
        x_center, y_center, w, h = map(float, parts[1:5])
        keypoints = list(map(float, parts[5:]))  # 나머지 부분은 키포인트

        # 바운딩 박스 좌표 복원
        x_center_abs = x_center * img_width
        y_center_abs = y_center * img_height
        w_abs = w * img_width
        h_abs = h * img_height

        x1 = int(x_center_abs - w_abs / 2)
        y1 = int(y_center_abs - h_abs / 2)
        x2 = int(x_center_abs + w_abs / 2)
        y2 = int(y_center_abs + h_abs / 2)

        # 바운딩 박스 그리기
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)  # 파란색 박스

        # 키포인트 그리기
        if len(keypoints) % 2 != 0:
            print(f"키포인트 수가 홀수입니다: {keypoints}")
            continue

        for i in range(0, len(keypoints), 2):
            x_norm = keypoints[i]
            y_norm = keypoints[i + 1]
            x_abs = int(x_norm * img_width)
            y_abs = int(y_norm * img_height)
            cv2.circle(image, (x_abs, y_abs), 3, (0, 255, 0), -1)  # 초록색 점

    # 이미지 표시
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [ ]:
images_path = Path('./datasets/infrared_of/label/image')
labels_path = Path('./datasets/infrared_of/label/json')
images_path = [f for f in images_path.iterdir()]
labels_path = labels_path.glob('*.txt')

for image_path, label_path in zip(images_path, labels_path):
    label_visualize(image_path, label_path)


In [ ]:
import os
import random

# Path to the images directory
path = './asd/test/images'

# List all files in the directory
all_files = os.listdir(path)

# Supported image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif']

# Filter out image files based on their extensions
image_files = [
    file for file in all_files
    if os.path.splitext(file)[1].lower() in image_extensions
]

# Determine the number of images to select
num_images_to_select = min(100, len(image_files))

# Randomly select images
random_images = random.sample(image_files, num_images_to_select)

# Output the full paths of the selected images
for img in random_images:
    model1.predict(os.path.join(path, img), save=True)


In [ ]:
import os
import random

# Path to the images directory
path = './asd/test/images'

# List all files in the directory
all_files = os.listdir(path)

# Supported image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif']

# Filter out image files based on their extensions
image_files = [
    file for file in all_files
    if os.path.splitext(file)[1].lower() in image_extensions
]

# Determine the number of images to select
num_images_to_select = min(100, len(image_files))

# Randomly select images
random_images = random.sample(image_files, num_images_to_select)

# Output the full paths of the selected images
for img in random_images:
    model2.predict(os.path.join(path, img), save=True, )


In [ ]:
model2.predict('./2024-10-24 173519.png', save=True)


In [ ]:
from tqdm.auto import tqdm
from pathlib import Path
from ultralytics import YOLO

path_auto = Path(f'./checkpoints/best.pt')

model_auto = YOLO(path_auto)

results = model_auto.predict('./datasets/vid/feed_summery.mp4', stream=True, save=True, verbose=False)
for result in tqdm(results, desc="Progress", mininterval=0.01):
    pass


In [ ]:
from tqdm.auto import tqdm
from pathlib import Path
from ultralytics import YOLO

path_auto = Path(f'./checkpoints/best.pt')
model_auto = YOLO(path_auto)

results = model_auto.predict('./datasets/vid/dinner_summery.mp4', stream=True, save=True, verbose=False)
for result in tqdm(results, desc="Progress", mininterval=0.01):
    pass


# ppt


In [ ]:
import cv2
import csv
import time
import mimetypes
import numpy as np

# Import YOLO for object detection / YOLO 객체 검출을 위한 임포트
from ultralytics import YOLO

from abc import ABCMeta, abstractmethod
from enum import Enum
from tqdm.auto import tqdm
from pathlib import Path
from typing import List, Tuple, Union, Optional

model1 = YOLO('./best1.pt')
model2 = YOLO('./best2.pt')


In [ ]:
def checkZeroKeypoints(keypoint: List[int]) -> bool:
    if keypoint == [0, 0]:
        return False
    return True


def drawBbox(image: np.ndarray, class_name: str, conf: float, bbox: Tuple[float, float, float, float],  width: Union[float, int], height: Union[float, int]) -> None:
    x1, y1, x2, y2 = list(map(int, bbox))

    # Draw rectangle and label text / 사각형과 라벨 텍스트 그리기
    if width == None and height == None:
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 255), 5, cv2.LINE_AA)
        cv2.putText(image, f'{class_name}:{conf:.2f}% Size estimation not possible', (x1, y1-30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
    elif width == None:
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 255), 5, cv2.LINE_AA)
        cv2.putText(image, f'{class_name}:{conf:.2f}% | H:{height:.2f}', (x1, y1-30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
    elif height == None:
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 255), 5, cv2.LINE_AA)
        cv2.putText(image, f'{class_name}:{conf:.2f}% | W:{width:.2f}', (x1, y1-30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
    else:
        cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 255), 5, cv2.LINE_AA)
        cv2.putText(image, f'{class_name}:{conf:.2f}% | W:{width:.2f} H:{height:.2f}', (x1, y1-30), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)


def drawPoint(image: np.ndarray, keypoints: List[Tuple[float, float]]) -> None:
    for kp in keypoints:
        kp = list(map(int, kp))

        # Only Draw if points are valid (not (0, 0))/ 키포인트가 유효한 경우에만 그리기 (0, 0 아님)
        if checkZeroKeypoints(kp):
            cv2.line(image, (kp), (kp), (0, 0, 255), 15)


def drawLine(image: np.ndarray, keypoints: List[Tuple[float, float]]) -> None:
    kp1, kp2, kp3, kp4 = keypoints

    # Only Draw if points are valid (not (0, 0))/ 키포인트가 유효한 경우에만 그리기 (0, 0 아님)
    if checkZeroKeypoints(kp1) and checkZeroKeypoints(kp2):
        # Draw lines between specific keypoints / 특정 키포인트 사이에 선 그리기
        cv2.line(image, (list(map(int, kp1))), (list(map(int, kp2))), (0, 255, 0), 5)

    # Only Draw if points are valid (not (0, 0))/ 키포인트가 유효한 경우에만 그리기 (0, 0 아님)
    if checkZeroKeypoints(kp3) and checkZeroKeypoints(kp4):
        # Draw lines between specific keypoints / 특정 키포인트 사이에 선 그리기
        cv2.line(image, (list(map(int, kp3))), (list(map(int, kp4))), (0, 255, 0), 5)


def ConvertCM(keypoints: List[Tuple[float, float]]) -> Tuple[float, float]:
    kp1, kp2, kp3, kp4 = keypoints
    kpx, kpy = list(zip(kp1, kp2, kp3, kp4))

    # Calculate width and height in cm using correction factor / 보정 값을 사용해 cm 단위로 폭과 높이 계산
    width = (((kpx[0]-kpx[1])**2 + (kpy[0]-kpy[1])**2)**0.5)
    height = (((kpx[2]-kpx[3])**2 + (kpy[2]-kpy[3])**2)**0.5)

    # Only width and height output if points are valid (not (0, 0))/ 키포인트가 유효한 경우에만 가로 세로 길이 출력 (0, 0 아님)
    if not checkZeroKeypoints(kp1) or not checkZeroKeypoints(kp2):
        width = 0
    if not checkZeroKeypoints(kp3) or not checkZeroKeypoints(kp4):
        height = 0
    return width, height


def writeToCSV(csv_path: Union[str, Path], keypoints: list, width: float, height: float) -> None:
    data = {
        'Keypoints': keypoints,
        'Width': width,
        'Height': height,
    }

    # Convert to Path object / Path 객체로 변환
    csv_path = Path(csv_path)

    # Check if file exists / 파일이 존재하는지 확인
    file_exists = csv_path.exists()

    with csv_path.open(mode="a", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=data.keys())

        if not file_exists:
            # Write header if file does not exist / 파일이 존재하지 않으면 헤더 작성
            writer.writeheader()

        # Write the data row / 데이터 행 작성
        writer.writerow(data)


In [ ]:
imgs = ['./swim_of_2021-01-12-09-02_24-31_0556.jpg', './swim_of_2021-01-27-09-00_00-47-18_506.jpg']

for image in imgs:
    csv_path = Path('./runs/pose/predict3') / "output.csv"
    img = cv2.imread(image)
    img_height, img_width, _ = img.shape
    print(img_height, img_width)
    result1 = model1.predict(img, stream=True)

    for result in result1:
        for r in result:
            keypoints = r.keypoints.xy.squeeze().tolist()
            bboxes = r.boxes.xyxy.squeeze().tolist()
            conf = r.boxes.conf.squeeze().tolist() * 100
            class_number = int(r.boxes.cls)

            drawPoint(img, keypoints)

            width, height = ConvertCM(keypoints)
            drawLine(img, keypoints)

            drawBbox(img, 'Olive-Flounder', conf, bboxes, width, height)

            writeToCSV(csv_path, keypoints, width, height)

        cv2.imwrite(f'./runs/pose/predict3/{image}', img)

for image in imgs:
    csv_path = Path('./runs/pose/predict4') / "output.csv"
    img = cv2.imread(image)
    img_height, img_width, _ = img.shape
    print(img_height, img_width)
    result2 = model2.predict(img, stream=True)

    for result in result2:
        for r in result:
            keypoints = r.keypoints.xy.squeeze().tolist()
            bboxes = r.boxes.xyxy.squeeze().tolist()
            conf = r.boxes.conf.squeeze().tolist() * 100
            class_number = int(r.boxes.cls)

            width, height = ConvertCM(keypoints)

            drawBbox(img, 'Olive-Flounder', conf, bboxes, width, height)

            writeToCSV(csv_path, keypoints, width, height)

        cv2.imwrite(f'./runs/pose/predict4/{image}', img)


In [ ]:
csv_path = Path('./runs/pose/predict3') / "output.csv"
img = cv2.imread('./swim_of_2021-01-27-09-00_00-47-18_506.jpg')
img_height, img_width, _ = img.shape
print(img_height, img_width)
result1 = model1.predict(img, stream=True)

for result in result1:
    for r in result:
        keypoints = r.keypoints.xy.squeeze().tolist()
        bboxes = r.boxes.xyxy.squeeze().tolist()
        conf = r.boxes.conf.squeeze().tolist() * 100
        class_number = int(r.boxes.cls)

        width, height = ConvertCM(keypoints)

        drawBbox(img, 'Olive-Flounder', conf, bboxes, width, height)

        writeToCSV(csv_path, keypoints, width, height)

    cv2.imwrite(f'./runs/pose/predict3/{image}', img)


In [ ]:
csv_path = Path('./runs/pose/predict4') / "output.csv"
img = cv2.imread('./swim_of_2021-01-12-09-02_24-31_0556.jpg')
img_height, img_width, _ = img.shape
print(img_height, img_width)
result2 = model2.predict(img, stream=True)

for result in result2:
    for r in result:
        keypoints = r.keypoints.xy.squeeze().tolist()
        bboxes = r.boxes.xyxy.squeeze().tolist()
        conf = r.boxes.conf.squeeze().tolist() * 100
        class_number = int(r.boxes.cls)

        width, height = ConvertCM(keypoints)

        drawBbox(img, 'Olive-Flounder', conf, bboxes, width, height)

        writeToCSV(csv_path, keypoints, width, height)

    cv2.imwrite(f'./runs/pose/predict4/{image}', img)


In [ ]:
import plotly.express as px
img = cv2.imread('./runs/pose/predict4/swim_of_2021-01-12-09-02_24-31_0556.jpg')

key = [
    [[2484.141357421875, 1415.259521484375], [2254.3330078125, 364.6940002441406], [2112.6298828125, 975.5111083984375], [2553.710205078125, 869.6729736328125]],
    [[1678.2811279296875, 921.1973266601562], [802.283447265625, 297.8110656738281], [1164.6920166015625, 818.0661010742188], [1386.2689208984375, 514.0041198730469]],
    [[1746.9703369140625, 611.9508056640625], [0.0, 0.0], [1241.77880859375, 441.853271484375], [1500.9075927734375, 158.32139587402344]],
    [[0.0, 0.0], [668.1292724609375, 395.08270263671875], [0.0, 0.0], [13.53026580810547, 303.6925964355469]],
    [[0.0, 0.0], [406.8952941894531, 1123.32568359375], [20.259164810180664, 602.420654296875], [0.0, 0.0]],
]
for i in key:
    drawPoint(img, i)
    width, height = ConvertCM(keypoints)
    drawLine(img, i)
    drawBbox(img, 'Olive-Flounder', conf, bboxes, width, height)

cv2.imwrite('./runs/pose/predict4/swim_of_2021-01-12-09-02_24-31_0556.jpg', img)
px.imshow(img)


In [ ]:
import plotly.express as px
img = cv2.imread('./runs/pose/predict4/swim_of_2021-01-27-09-00_00-47-18_506.jpg')

key = [
    [[376.75299072265625, 872.4286499023438], [1402.9210205078125, 259.89004516601562], [722.1155395507812, 403.591064453125], [963.8826293945312, 791.8745727539062]],
    [[0.0, 0.0], [1637.4248046875, 1157.677001953125], [2344.576904296875, 1223.2137451171875], [2233.428955078125, 796.3973999023438]],
    [[197.41160583496094, 706.7562866210938], [0.0, 0.0], [334.54473876953125, 124.57510375976562], [698.9071044921875, 389.922607421875]],
    [[0.0, 0.0], [2524.03173828125, 768.9278564453125], [0.0, 0.0], [2589.718994140625, 313.65945434570312]]
]
for i in key:
    drawPoint(img, i)
    drawLine(img, i)

cv2.rectangle(img, (168, 0), (930, 705), (255, 0, 255), 5, cv2.LINE_AA)
cv2.imwrite('./runs/pose/predict4/swim_of_2021-01-27-09-00_00-47-18_506.jpg', img)
px.imshow(img)


# SAM2


In [ ]:
from ultralytics.data.annotator import auto_annotate

auto_annotate(data="./datasets/infrared_of/train", det_model="yolov8n.pt", sam_model="sam2_b.pt")


In [ ]:
import cv2
from pathlib import Path

image_path = Path('./datasets/infrared_of/train/')
label_path = Path('./datasets/infrared_of/train_auto_annotate_labels')


In [ ]:
labels = [f for f in label_path.iterdir() if f.is_file()]
images = [f for f in image_path.iterdir() for l in labels if f.stem == l.stem]


In [ ]:
img_lst = []
for image, label in list(zip(images, labels)):
    img = cv2.imread(image)
    h, w, _ = img.shape

    with label.open(mode='r', encoding='utf-8') as f:
        content = f.readlines()

    tokens = content.split()

    coords = [float(token) for token in tokens[1:]]

    points = list(zip(coords[::2], coords[1::2]))

    for i in points:
        x, y = int(i[0] * (w//2)), int(i[1] * (h//2))
        img = cv2.circle(img, (x, y), 5, (0, 0, 255), -1)
        img = cv2.resize(img, ((w//2), h//2))

    img_lst.append(img)


In [ ]:
for img in img_lst:
    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

# Initialize a YOLO-World model
model = YOLO("yolov8s-world.pt")  # or select yolov8m/l-world.pt

# Define custom classes
model.set_classes(["fish"])

# Save the model with the defined offline vocabulary
model.save("custom_yolov8s.pt")


In [ ]:
label_path = Path('./datasets/infrared_of/train_auto_annotate_labels')
images = [f for f in image_path.iterdir() if f.is_file()]
for image in images:
    img = cv2.imread(image)
    h, w, _ = img.shape
    img = cv2.resize(img, ((w//2), h//2))
    cv2.imshow('img', img)
    cv2.waitKey()
    cv2.destroyAllWindows()


In [ ]:
auto_labeling = YOLO('./checkpoints/best.pt')

path = Path('./datasets/infrared_of/test/')
results = auto_labeling.predict(path, stream=True, save=True, save_txt=True)
for r in results:
    print(r.keypoints.conf)
